In [0]:
df_customers = spark.read.option("header", True).option("inferSchema","true").csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/customers.csv")

df_products = spark.read.option("header", True).csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/products.csv")


df_categories = spark.read.option("header", True).csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/category.csv")

df_brands = spark.read.option("header", True).csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/brands.csv")

df_date = spark.read.option("header", True).csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/date.csv")


In [0]:
df_customers.write.format("delta").mode("overwrite").saveAsTable("workspace.ecommerce_sample_raw.bronze_customers")

In [0]:
df_products.write.format("delta").mode("overwrite").saveAsTable("workspace.ecommerce_sample_raw.bronze_products")
df_categories.write.format("delta").mode("overwrite").saveAsTable("workspace.ecommerce_sample_raw.bronze_category")
df_brands.write.format("delta").mode("overwrite").saveAsTable("workspace.ecommerce_sample_raw.bronze_brands")
df_date.write.format("delta").mode("overwrite").saveAsTable("workspace.ecommerce_sample_raw.bronze_date")


In [0]:
%sql
-- Step 1: Tell the table to prioritize 'product_id'
ALTER TABLE workspace.ecommerce_sample_raw.bronze_products 
CLUSTER BY (product_id);

-- Step 2: Rewrite the existing files to follow that new rule
OPTIMIZE workspace.ecommerce_sample_raw.bronze_products;

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 0, false, 0, 0, 1768274153705, 1768274158958, 8, 0, null, List(0, 0), null, 12, 12, 0, 0, List(625635, true, false, false, null, null, null, null, 0, 0, 0, 0, 1, 625635, 625635, null, log, 16777216, 67108864, 4, 0, 0, null, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(137, 71, 0, 0, 0, 2663), 2, 1, 5, sizeAware))"
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1768274159011, 1768274161207, 8, 0, null, List(0, 0), null, 12, 12, 0, 0, List(625635, false, false, false, null, null, null, post-optimize-compaction, 0, 0, 0, 0, 0, 0, 0, null, null, 33554432, 67108864, 0, 0, 0, null, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(0, 0, 947, 0, 0, 0), 15, 1, 1, null))"


In [0]:
%sql
DESCRIBE HISTORY workspace.ecommerce_sample_raw.bronze_products;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2026-01-13T03:15:58.000Z,70692347892489,jayaraniarunachalam@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [""product_id""], isFull -> false, zOrderBy -> [], batchId -> -1)",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,3,SnapshotIsolation,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
3,2026-01-13T03:15:53.000Z,70692347892489,jayaraniarunachalam@gmail.com,CLUSTER BY,"Map(oldClusteringColumns -> , newClusteringColumns -> product_id)",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,2,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
2,2026-01-13T03:15:52.000Z,70692347892489,jayaraniarunachalam@gmail.com,ROW TRACKING BACKFILL,Map(batchId -> 0),null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,1,SnapshotIsolation,false,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-13T03:15:51.000Z,70692347892489,jayaraniarunachalam@gmail.com,UPGRADE PROTOCOL,"Map(newProtocol -> {""minReaderVersion"":3,""minWriterVersion"":7,""readerFeatures"":[""deletionVectors""],""writerFeatures"":[""deletionVectors"",""domainMetadata"",""rowTracking"",""invariants"",""appendOnly""]})",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,0,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-13T03:03:16.000Z,70692347892489,jayaraniarunachalam@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 50000, numOutputBytes -> 625635)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


### Create empty Bronze orders table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.ecommerce_sample_raw.bronze_orders (
  dt DATE,
  order_ts TIMESTAMP,
  customer_id STRING,
  order_id STRING,
  item_seq INT,
  product_id STRING,
  quantity INT,
  unit_price_currency STRING,
  unit_price DOUBLE,
  discount_pct STRING,
  tax_amount DOUBLE,
  channel STRING,
  coupon_code STRING,
  updated_at TIMESTAMP
)
USING DELTA;


In [0]:
%sql
SELECT * FROM workspace.ecommerce_sample_raw.bronze_orders;

dt,order_ts,customer_id,order_id,item_seq,product_id,quantity,unit_price_currency,unit_price,discount_pct,tax_amount,channel,coupon_code,updated_at


### Read new order CSVs (incremental)

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
user_schema = StructType([
    StructField("dt", DateType(), True),
    StructField("order_ts", TimestampType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("item_seq", IntegerType(), True),
    StructField("product_id", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("unit_price_currency", StringType(), True),
    StructField("unit_price", DoubleType(), True),
    StructField("discount_pct", StringType(), True),
    StructField("tax_amount", DoubleType(), True),
    StructField("channel", StringType(), True),
    StructField("coupon_code", StringType(), True)
])

df_orders = (
    spark.read
    .option("header", True)
    .schema(user_schema)
    .csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/order_items_2025-08-01.csv")
    .withColumn("updated_at", current_timestamp())
)
df_orders.createOrReplaceTempView("orders_staging")

In [0]:
df_orders.printSchema()

root
 |-- dt: date (nullable = true)
 |-- order_ts: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- item_seq: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price_currency: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- discount_pct: string (nullable = true)
 |-- tax_amount: double (nullable = true)
 |-- channel: string (nullable = true)
 |-- coupon_code: string (nullable = true)
 |-- updated_at: timestamp (nullable = false)



### MERGE (Upsert Logic)

In [0]:
%sql
MERGE INTO workspace.ecommerce_sample_raw.bronze_orders t
USING orders_staging s
ON t.order_id = s.order_id
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1469,0,0,1469


In [0]:
%sql
SELECT * FROM workspace.ecommerce_sample_raw.bronze_orders LIMIT(20);

dt,order_ts,customer_id,order_id,item_seq,product_id,quantity,unit_price_currency,unit_price,discount_pct,tax_amount,channel,coupon_code,updated_at
2025-08-01,2025-08-01T22:53:52.000Z,CUST000000241190,643611,1,2000000028279,1,GBP,11.0,10%,2.0,web,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T22:53:52.000Z,CUST000000241190,643611,2,2000000377445,1,GBP,5.0,4%,1.0,web,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T09:44:32.000Z,CUST000000239553,643612,1,2000000417639,4,INR,4871.0,8%,3245.0,web,FEST20,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T04:45:03.000Z,CUST000000175269,643613,1,2000000422664,null,AED,74.0,0%,18.0,app,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T04:45:03.000Z,CUST000000175269,643613,2,2000000238159,1,AED,1012.0,11%,109.0,app,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T04:45:03.000Z,CUST000000175269,643613,3,2000000094205,null,AED,229.0,15%,20.0,app,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T04:45:03.000Z,CUST000000175269,643613,4,2000000293790,null,AED,2222.0,0%,801.0,app,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T21:09:10.000Z,CUST000000117094,643614,1,2000000271415,3,INR,252.0,10%,82.0,app,null,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T13:03:57.000Z,CUST000000016826,643615,1,2000000050980,1,GBP,71.0,5%,9.0,web,SAVE50,2026-01-13T04:16:03.400Z
2025-08-01,2025-08-01T17:54:51.000Z,CUST000000252796,643616,1,2000000463254,1,USD,null,9%,1.0,app,null,2026-01-13T04:16:03.400Z


## TIME TRAVEL

### Check History

In [0]:
%sql
DESCRIBE HISTORY workspace.ecommerce_sample_raw.bronze_products;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2026-01-13T03:15:58.000Z,70692347892489,jayaraniarunachalam@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [""product_id""], isFull -> false, zOrderBy -> [], batchId -> -1)",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,3,SnapshotIsolation,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
3,2026-01-13T03:15:53.000Z,70692347892489,jayaraniarunachalam@gmail.com,CLUSTER BY,"Map(oldClusteringColumns -> , newClusteringColumns -> product_id)",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,2,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
2,2026-01-13T03:15:52.000Z,70692347892489,jayaraniarunachalam@gmail.com,ROW TRACKING BACKFILL,Map(batchId -> 0),null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,1,SnapshotIsolation,false,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-13T03:15:51.000Z,70692347892489,jayaraniarunachalam@gmail.com,UPGRADE PROTOCOL,"Map(newProtocol -> {""minReaderVersion"":3,""minWriterVersion"":7,""readerFeatures"":[""deletionVectors""],""writerFeatures"":[""deletionVectors"",""domainMetadata"",""rowTracking"",""invariants"",""appendOnly""]})",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,0,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-13T03:03:16.000Z,70692347892489,jayaraniarunachalam@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)",null,List(2657594677920203),0113-025118-q0oc8dgd-v2n,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 50000, numOutputBytes -> 625635)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


### Query Previous Version

In [0]:
%sql
SELECT COUNT(*) FROM workspace.ecommerce_sample_raw.bronze_products VERSION AS OF 2;


COUNT(*)
50000


### Reading orders csv of 02-08-2025 and merge (upsert) with bronze_orders

In [0]:
df_orders = (
    spark.read
    .option("header", True)
    .schema(user_schema)
    .csv("/Volumes/workspace/ecommerce_sample_raw/ecommerce_sample_raw_volume/order_items_2025-08-02.csv")
    .withColumn("updated_at", current_timestamp())
)

In [0]:
from delta.tables import DeltaTable
target = DeltaTable.forName(spark, "workspace.ecommerce_sample_raw.bronze_orders")

target.alias("t").merge(
    df_orders.alias("s"),
    "t.order_id = s.order_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
DESCRIBE HISTORY workspace.ecommerce_sample_raw.bronze_orders;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2026-01-13T04:34:45.000Z,70692347892489,jayaraniarunachalam@gmail.com,MERGE,"Map(predicate -> [""(order_id#16811 = order_id#16780)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> true, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 40737, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 2330, materializeSourceTimeMs -> 551, numTargetRowsInserted -> 1848, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 624, numTargetRowsUpdated -> 0, numOutputRows -> 1848, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1848, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1063)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-13T04:16:07.000Z,70692347892489,jayaraniarunachalam@gmail.com,MERGE,"Map(predicate -> [""(order_id#15175 = order_id#15141)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> true, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 32709, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 2370, materializeSourceTimeMs -> 381, numTargetRowsInserted -> 1469, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 512, numTargetRowsUpdated -> 0, numOutputRows -> 1469, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1469, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1375)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-13T03:54:33.000Z,70692347892489,jayaraniarunachalam@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true"",""delta.writePartitionColumnsToParquet"":""true"",""delta.enableRowTracking"":""true"",""delta.rowTracking.materializedRowCommitVersionColumnName"":""_row-commit-version-col-09cb6a6a-e716-4caa-95b3-b959d98dd957"",""delta.rowTracking.materializedRowIdColumnName"":""_row-id-col-9cb2c412-7f58-4848-9fcd-a990477b8c81""}, statsOnLoad -> false)",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,null,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13


In [0]:
%sql
SELECT COUNT(*) FROM workspace.ecommerce_sample_raw.bronze_orders VERSION AS OF 1;

COUNT(*)
1469


### comparing Data Drift

In [0]:
%sql
SELECT COUNT(*) FROM workspace.ecommerce_sample_raw.bronze_orders VERSION AS OF 2;

COUNT(*)
3317


###  comparing the count of rows in the above two version we can conclude that all data from source is merged with target and Delta Lake automatically keeps versions of your table every time data is INSERT, UPDATE, DELETE, MERGE

### optimizing bronze_orders to improve query performance

In [0]:
spark.sql("OPTIMIZE workspace.ecommerce_sample_raw.bronze_orders ZORDER BY (order_id)")
spark.sql("VACUUM workspace.ecommerce_sample_raw.bronze_orders RETAIN 168 HOURS")


DataFrame[path: string]

In [0]:
%sql
DESCRIBE HISTORY workspace.ecommerce_sample_raw.bronze_orders;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2026-01-13T04:47:23.000Z,70692347892489,jayaraniarunachalam@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(2657594677920203),0113-035406-1pwio5ym-v2n,4,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
4,2026-01-13T04:47:22.000Z,70692347892489,jayaraniarunachalam@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000, specifiedRetentionMillis -> 604800000)",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,3,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
3,2026-01-13T04:47:14.000Z,70692347892489,jayaraniarunachalam@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> false, clusterBy -> [], zOrderBy -> [""order_id""], batchId -> 0)",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,2,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 73446, p25FileSize -> 69600, numDeletionVectorsRemoved -> 0, minFileSize -> 69600, numAddedFiles -> 1, maxFileSize -> 69600, p75FileSize -> 69600, p50FileSize -> 69600, numAddedBytes -> 69600)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
2,2026-01-13T04:34:45.000Z,70692347892489,jayaraniarunachalam@gmail.com,MERGE,"Map(predicate -> [""(order_id#16811 = order_id#16780)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> true, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 40737, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 2330, materializeSourceTimeMs -> 551, numTargetRowsInserted -> 1848, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 624, numTargetRowsUpdated -> 0, numOutputRows -> 1848, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1848, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1063)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
1,2026-01-13T04:16:07.000Z,70692347892489,jayaraniarunachalam@gmail.com,MERGE,"Map(predicate -> [""(order_id#15175 = order_id#15141)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> true, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(2657594677920203),0113-035406-1pwio5ym-v2n,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 32709, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 2370, materializeSourceTimeMs -> 381, numTargetRowsInserted -> 1469, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 512, numTargetRowsUpdated -> 0, numOutputRows -> 1469, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1469, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1375)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
0,2026-01-13T03:54:33.000Z,70692347892489,jayaraniarunachalam@gmail.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true"",""delta.writePartitionColumnsToParquet"":""true"",""delta.enableRo